Метод **многоугловой дифференциальной спектроскопии** (MAX DOAS – Multi-Axis Differential Absorbtion Spectroscopy) использует измерения спектра рассеянного солнечного излучения в полосе поглощения исследуемого газа ($\text{NO}_2$). Измерения спектров приходящего излучения выполняются из направлений с различными зенитными углами.

Мы рассматриваем измерения при фиксированном азимуте относительно положения солнца. Также предполагается, что атмосфера сферически симметрична. С использованием так называемой **процедуры DOAS анализа** определяется наклонное содержание газа ($\text{NO}_2$) $S(z)$ при каждом направлении визирования $z$.

$$\pmb\xi = M \pmb n,$$
$$M: V \rightarrow W; \, 
M(\pmb n_1 + \pmb n_2) = M(\pmb n_1) + M(\pmb n_2), \,
M(c \pmb n) = c M(\pmb n),$$
$$\dim V = d_1, \, \dim W = d_2.$$

$$||\pmb \xi|| = \max_{i=1, \ldots, d_2} |\xi_i|$$

Пусть линейный оператор задан матрицей $||m^p_k||^{d_2}_{d_1}$, 
строки которой обозначим $m_i, \, i = \overline{1, d_2}$. $\pmb \xi$ соотвествует
$||\xi^p||^{d_2}$, $\pmb n$ соотвествует $||n^p||^{d_1}$.
Тогда:

$$\xi_i = (m_i^T, \pmb n), \quad i = \overline{1, d_2}$$

Рассмотрим задачу на минимакс

$$\min_{\pmb n}\max_{i=\overline{1, d_2}} |\xi_i - (m_i^T, \pmb n)|$$

$$|d| = \min_{z} \{ z \, | \, z \geq d \wedge z \geq -d\}$$

Соответственно:

$$\max_{i=\overline{1, d_2}} |\xi_i - (m_i^T, \pmb n)| = 
\min_{z} \{z \,| \, z \geq \xi_i - (m_i^T, \pmb n) \wedge z \geq -\xi_i + (m_i^T, \pmb n)\}
, \quad i = \overline{1, d_2}$$

Введём два вектора $s = (z, n_1, \ldots, n_{d_1})$ и $t = (1, 0, \ldots, 0)$. 
Их скалярное произведение$(s, t) = z$.

Минимизация по $s$ при наложенных ограничениях даст решение искомой задачи:

$$\min_s \{(s, t) \, | \,  z \geq \xi_i - (m_i^T, \pmb n) \wedge z \geq -\xi_i + (m_i^T, \pmb n)\}
, \, i = \overline{1, d_2} \}\, \Longrightarrow \, s$$ 

$+$ наличие априорной информации(унимодальность профиля и неотрицательность концентраций)

$$\exists k \in {1, \ldots, d_1}:
\begin{cases}
n_1 \leq n_2, \\
\ldots \\
n_{k-1} \leq n_{k}, \\
n_{k} \geq n_{k+1}, \\
\ldots \\
n_{d_1 - 1} \geq n_{d_1}.
\end{cases}
\qquad and \qquad \forall i \;  (n_i \geq 0).$$

In [1]:
import sys
import csv
import matplotlib.pyplot as plt
import numpy as np
from math import fabs, sqrt
from pathlib import Path
sys.path.insert(1, '../scripts')

from profile_recovery import ProfileRec

In [2]:
dict_profile_time = {
    ("06", "42"): "PM004A",
    ("07", "05"): "PM004D",
    ("07", "37"): "PM005A",
    ("08", "08"): "PM005D",
    ("09", "42"): "PM006A",
    ("10", "13"): "PM006D",
    ("11", "01"): "PM007A",
    ("11", "27"): "PM007D",
    ("13", "06"): "PM008A",
    ("13", "23"): "PM008D",
    ("16", "54"): "PM010A",
    ("17", "17"): "PM010D",
    ("17", "46"): "PM011A",
    ("18", "04"): "PM011D",
}

In [3]:
name_height = {
    "unimod_1.csv": 0,
    "unimod_2.csv": 300,
    "unimod_3.csv": 800,
    "threemod_1.csv": 100,
    "threemod_2.csv": 600,
}

In [4]:
def generate_bars(heights, p_x, p_y):
    def integrate(list_h, list_n):
        sum_n = 0
        for i in range(len(list_h) - 1):
            sum_n += 0.5 * (list_h[i] - list_h[i + 1]) * (list_n[i] + list_n[i + 1])
        return sum_n / (list_h[0] - list_h[-1])

    fin_h = []
    fin_n = []
    dict_index = {0.0: len(p_y)}
    for h in heights:
        for i, x in enumerate(p_x):
            if h >= x:
                dict_index[h] = i
                break
    keys = list(dict_index.keys())
    for h in range(len(keys) - 1):
        fin_h.append(keys[h])
        fin_n.append(
            integrate(
                p_x[dict_index[keys[h + 1]] : dict_index[keys[h]]],
                p_y[dict_index[keys[h + 1]] : dict_index[keys[h]]],
            )
        )
    return fin_h, fin_n

In [5]:
def build(sigma: float, number_of_trials: int, profiles: str, dataset: str) -> None:                                                                               
    path_to_profiles = Path(Path.cwd().parents[1], "data", profiles, "csv")                                                                                    
    k: int = 1                                                                                                                                                     
    for txt_path in sorted(list(path_to_profiles.glob("*.csv"))):                                                                                                  
        if txt_path.name == "profile_06_42.csv":                                                                                                                   
            continue                                                                                                                                               
        sp = plt.subplot(2, 2, k)                                                                                                                                  
        prof_recovery = ProfileRec(txt_path.name, profiles, dataset)                                                                                               
        rec = prof_recovery.linear_programming(sigma)                                                                                                              
        rec_min_max = [[x, x] for x in rec[1]]                                                                                                                     
        rec_avg = [0] * len(rec[1])                                                                                                                                
        rec_list = []                                                                                                                                              
        step = rec[0][1] - rec[0][0]                                                                                                                               
        for _ in range(number_of_trials):                                                                                                                          
            rec = prof_recovery.linear_programming(sigma)                                                                                                          
            for i in range(len(rec[1])):                                                                                                                           
                rec_avg[i] += rec[1][i]                                                                                                                            
            rec_list.append(rec[1])                                                                                                                                
        plt.errorbar(                                                                                                                                              
            [x + 0.5 * step for x in rec[0]],                                                                                                                      
            [r / number_of_trials for r in rec_avg],                                                                                                               
            yerr=[np.std([x[i] * 0.5 for x in rec_list]) for i in range(len(rec[1]))],                                                                             
            fmt="o",                                                                                                                                               
            ecolor="black",                                                                                                                                        
            elinewidth=2.5,                                                                                                                                        
            capsize=4,                                                                                                                                             
            color="black",                                                                                                                                         
            label=f"average restored n={number_of_trials}"                                                                                                         
        )                                                                                                                                                          
        list_discrepancy = list()                                                                                                                                  
        step = rec[0][1] - rec[0][0]                                                                                                                               
        plt.bar(                                                                                                                                                   
            [h + 0.5 * step for h in rec[0]],                                                                                                                      
            rec[1],                                                                                                                                                
            width=step,                                                                                                                                            
            linewidth=1,                                                                                                                                           
            edgecolor="black",                                                                                                                                     
            alpha=0.5,                                                                                                                                             
            color="red",                                                                                                                                           
            linestyle="-",                                                                                                                                         
            label="one restored profile",                                                                                                                          
        )                                                                                                                                                          
        with open(txt_path, "r") as csv_file:                                                                                                                      
            reader = csv.reader(csv_file)                                                                                                                          
            tup_coords: tuple[tuple[str]] = tuple(reader)                                                                                                          
            plt.minorticks_on()                                                                                                                                    
            plt.grid(which="major", color="k", linewidth=1)                                                                                                        
            plt.grid(which="minor", color="k", linestyle=":")                                                                                                      
            plt.xlabel("h[m]")                                                                                                                                     
            plt.ylabel(r"$n$ [$cm^{-3}$]")                                                                                                                         
            plt.xlim(0, 10**3)                                                                                                                                     
            list_text = txt_path.name.replace(".csv", "").split("_")                                                                                               
            plt.title(                                                                                                                                             
                f"""{dict_profile_time.get(tuple(list_text[1:]))}: {list_text[1]}:{list_text[2]} [szas=60, wavesNM=450, albedo=0.05]"""                            
            )                                                                                                                                                      
            original_x = tuple(float(x[0]) for x in tup_coords)                                                                                                    
            original_y = tuple(float(x[1]) for x in tup_coords)                                                                                                    
            list_discrepancy.append((0, fabs(rec[1][0] - original_y[-1])))                                                                                         
            for res_i, res_h in enumerate(rec[0]):                                                                                                                 
                for orig_i, orig_h in enumerate(original_x):                                                                                                       
                    if res_h >= orig_h:                                                                                                                            
                        list_discrepancy.append(                                                                                                                   
                            (                                                                                                                                      
                                rec[0][res_i],                                                                                                                     
                                sqrt((rec[1][res_i] - original_y[orig_i]) ** 2),                                                                                   
                            )                                                                                                                                      
                        )                                                                                                                                          
                        break                                                                                                                                      
            data_bars = generate_bars(rec[0], original_x, original_y)                                                                                              
            # step = rec[0][1] - rec[0][0]                                                                                                                         
            plt.bar(                                                                                                                                               
                [h + 0.5 * step for h in data_bars[0]],                                                                                                            
                data_bars[1],                                                                                                                                      
                width=step,                                                                                                                                        
                linewidth=1,                                                                                                                                       
                alpha=0.5,                                                                                                                                         
                edgecolor="darkblue",                                                                                                                              
                color="turquoise",                                                                                                                                 
                linestyle="-",                                                                                                                                     
            )                                                                                                                                                      
            plt.errorbar(                                                                                                                                          
                original_x,                                                                                                                                        
                original_y,                                                                                                                                        
                linewidth=2,                                                                                                                                       
                label="original",                                                                                                                                  
            )                                                                                                                                                      
        plt.legend()                                                                                                                                               
        k += 1                                                                                                                                                     
        if k > 4:                                                                                                                                                  
            k = 1                                                                                                                                                  
            plt.figure()                                                                                                                                           
        break
    plt.figure()
    plt.show()                                                                                                                                                     

build(0.3, 10, "profiles_1", "dataset_2_csv")

<Figure size 432x288 with 0 Axes>